In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import datetime
import glob ,sys , re
from multiprocessing import Pool
import multiprocessing
from categorical_encoding import select_category_value_agg, select_level_agg, select_category_value_agg, cnt_encoding
from mv_wg_avg import exp_weight_avg
from target_encoding import target_encoding

sys.path.append('../../../github/module/')
from preprocessing import set_validation, split_dataset, get_dummies, factorize_categoricals, impute_avg
from convinience_function import get_categorical_features, get_numeric_features, row_number, pararell_process
from logger import logger_func
from feature_engineering import base_aggregation, diff_feature, division_feature
from make_file import make_npy, make_feature_set
from tqdm import tqdm

logger = logger_func()
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)

unique_id = 'SK_ID_CURR'
b_id = 'SK_ID'
p_id = 'SK_ID_PREV'
target = 'TARGET'
ignore_features = [unique_id, target, 'valid_no', 'is_train', 'is_test', 'SK_ID_PREV']

2018-07-24 07:43:18,805 logger 19 [INFO]    [logger_func] start 
2018-07-24 07:43:18,815 logger 19 [INFO]    [logger_func] start 
2018-07-24 07:43:18,815 logger 19 [INFO]    [logger_func] start 
2018-07-24 07:43:18,839 logger 19 [INFO]    [logger_func] start 
2018-07-24 07:43:18,839 logger 19 [INFO]    [logger_func] start 
2018-07-24 07:43:18,839 logger 19 [INFO]    [logger_func] start 


In [40]:
base = pd.read_csv('../data/base.csv')[unique_id].to_frame()
bureau = pd.read_csv('../data/bureau_after.csv')
bureau_clean = pd.read_csv('../data/bureau_cleansing.csv')
bureau.sort_values(by=[unique_id, 'SK_ID_BUREAU'], inplace=True)
bureau = row_number(bureau, unique_id)

train = pd.read_csv('../data/application_train_test.csv')
app = train[[unique_id, 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH']]
base_bureau = bureau[['SK_ID_CURR', 'SK_ID_BUREAU']].drop_duplicates()

' bureau_balanceで遅延したことのあるIDのみJOINする '
balance = pd.read_csv('../data/rawdata/bureau_balance.csv')
balance['STATUS'] =  balance['STATUS'].map(lambda x: '-1' if x=='C' or x=='X' else x).astype('int')
b_status = balance.groupby(b_id)['STATUS'].max()

b_status = b_status.reset_index().query('STATUS>=0')
bureau = bureau_clean.merge(b_status, on=b_id, how='left')

In [41]:
df = bureau
df['STATUS'] = df['STATUS'] + 1
df['DAYS_ENDDATE'] = df.apply(lambda x: x['DAYS_ENDDATE_FACT'] if x['DAYS_ENDDATE_FACT'] < 99999 else x['DAYS_CREDIT_ENDDATE'], axis=1)

' ENDDATE_FACT '
df['CREDIT_TERM_FACT'] = df['DAYS_ENDDATE_FACT'] - df['DAYS_CREDIT']
df['CREDIT_TERM_FACT'] = df['CREDIT_TERM_FACT'].map(lambda x:0 if x<0 else x)
df['LOAN_MONTHLY_FACT'] = df['AMT_CREDIT_SUM'] / df['CREDIT_TERM_FACT']

' ENDDATE '
df['CREDIT_TERM'] = df['DAYS_ENDDATE'] - df['DAYS_CREDIT']
df['CREDIT_TERM'] = df['CREDIT_TERM'].map(lambda x:0 if x<0 else x)
df['LOAN_MONTHLY'] = df['AMT_CREDIT_SUM'] / df['CREDIT_TERM']

In [42]:
amt_list = [col for col in bureau_clean.columns if col.count('AMT_') or col.count('LOAN_')]
ab = bureau.merge(app, on=unique_id, how='inner')
df = ab

for num in amt_list:
    df = division_feature(df=df, first=num, second='AMT_INCOME_TOTAL')   

In [10]:
def make_feature(df, prefix='b_'):
    result = base.merge(df, on=unique_id, how='left')
    make_npy(result=result, ignore_list=ignore_features, logger=logger, prefix=prefix)

In [1]:
' OVERDUEの数と割合 '
ab['BUREAU_OVERDUE_CNT_ALL@'] = ab['AMT_CREDIT_MAX_OVERDUE'].fillna(0) + ab['AMT_CREDIT_SUM_OVERDUE'].fillna(0) + ab['STATUS'].fillna(0)
ab['BUREAU_OVERDUE_CNT_ALL@'] = ab['BUREAU_OVERDUE_CNT_ALL@'].map(lambda x: 1 if x>0 else 0)
cnt_all = ab.groupby(unique_id)['BUREAU_OVERDUE_CNT_ALL@'].sum().reset_index()
cnt_all = ab.groupby(unique_id)['BUREAU_OVERDUE_CNT_ALL@'].sum().reset_index()
# cnt_all['BUREAU_OVERDUE_CNT_ALL@'].value_counts()
freq_overdue = cnt_all.merge(ab.groupby(unique_id).size().reset_index(), on=unique_id, how='inner').rename(columns={0:'CNT_ID@'})
freq_overdue['BUREAU_OVERDUE_ratio@'] = freq_overdue['BUREAU_OVERDUE_CNT_ALL@'] / freq_overdue['CNT_ID@']

freq_overdue = freq_overdue.merge(app, on=unique_id, how='inner')
num_list = ['BUREAU_OVERDUE_ratio@', 'CNT_ID@']
for num in num_list:
    freq_overdue = division_feature(df=freq_overdue, first=num, second='DAYS_BIRTH')
   
make_feature(df=freq_overdue, prefix='b_')

In [96]:
' 1年、半年、全期間のACTIVE, CLOSED, Bad debt, Sold, ID数 '
# prefix = 'b_YEAR_'
prefix = 'b_'

# act = 'SOLD'
# df = ab.query('DAYS_ENDDATE>=-12')
# df = ab.query("CREDIT_ACTIVE>='Sold'")
# df = ab
df['DAYS_ENDDATE'].fillna(999, inplace=True)

# cnt_act = df.groupby(unique_id).size().reset_index().rename(columns={0:f'CNT_{act}@'})
cnt_act = df.query('DAYS_ENDDATE>0').groupby(unique_id).size().reset_index().rename(columns={0:'CNT_ACTIVE@'})
cnt_id = df.groupby(unique_id).size().reset_index().rename(columns={0:'CNT_ID@'})

act_ratio = cnt_id.merge(cnt_act, on=unique_id, how='left').fillna(0)
act_ratio['CNT_CLOSED@'] = act_ratio['CNT_ID@'] - act_ratio['CNT_ACTIVE@']

act_ratio = division_feature(df=act_ratio, first='CNT_ACTIVE@', second='CNT_ID@')
# act_ratio = division_feature(df=act_ratio, first='CNT_SOLD@', second='CNT_ID@')
make_feature(act_ratio, prefix=prefix)

In [13]:
def extract_term(df, term):
    df = df.query(f'DAYS_CREDIT>={term}')
    return df

In [15]:
income_list = [col for col in ab.columns if col.count('INCOME') and col.count('@')]

' np.infとnp.nanは全て0にする '
for col in income_list:
    ab[col] = ab[col].map(lambda x : 0 if x==np.inf else x)
    ab[col] = ab[col].map(lambda x : 0 if x!=x else x)

In [2]:
prefix = 'b_'
data = ab.copy()
data['DAYS_ENDDATE'] = data['DAYS_ENDDATE'].where(data['DAYS_ENDDATE']<0, 0)
data['DAYS_ENDDATE'] = data['DAYS_ENDDATE'].map(lambda x: np.abs(x))

' 重み付き平均 '
label = 'DAYS_ENDDATE'
weight_list = [0.99, 0.95, 0.9]
feature_list = ['AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_diff_AMT_CREDIT_SUM_DEBT@' ,'AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'AMT_ANNUITY', 'LOAN_MONTHLY_FACT', 'LOAN_MONTHLY']
feature_list = income_list
days_list = ['DAYS_CREDIT', 'DAYS_CREDIT_UPDATE']
feature_list = feature_list + days_list

for num in feature_list:
    for weight in weight_list:
        wavg = exp_weight_avg(data=data, level=unique_id, weight=weight, label=label, value=num)
        result = base.merge(wavg.to_frame().reset_index(), on=unique_id, how='left').fillna(0)
        make_npy(logger=logger, result=result, ignore_list=ignore_features, prefix=prefix)
del data

In [17]:
amt_list

['AMT_CREDIT_MAX_OVERDUE',
 'AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_SUM_LIMIT',
 'AMT_CREDIT_SUM_OVERDUE',
 'AMT_ANNUITY']

In [18]:
data = ab
num_list = ['CREDIT_TERM', 'DAYS_CREDIT', 'DAYS_ENDDATE', 'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE']
select_level_agg(base=base, data=data, ignore_list=ignore_features, level=unique_id, method_list=method_list, num_list=num_list, prefix=prefix)

2018-07-21 15:36:05,017 logger 122 [INFO]    [select_level_agg] 
num: CREDIT_TERM
method: sum
['SK_ID_CURR'] 
2018-07-21 15:36:05,017 logger 122 [INFO]    [select_level_agg] 
num: CREDIT_TERM
method: sum
['SK_ID_CURR'] 
../../../github/module/feature_engineering.py:26: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  result = df.groupby(level)[feature].agg({'tmp': {method}})
2018-07-21 15:36:05,157 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'CREDIT_TERM_sum@['SK_ID_CURR']'], dtype='object') 
2018-07-21 15:36:05,157 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'CREDIT_TERM_sum@['SK_ID_CURR']'], dtype='object') 
2018-07-21 15:36:05,237 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 15:36:05,237 logger 130 [INFO]    [select_level_agg] 
result shape: (356255,

2018-07-21 15:36:05,550 logger 122 [INFO]    [select_level_agg] 
num: CREDIT_TERM
method: std
['SK_ID_CURR'] 
2018-07-21 15:36:05,550 logger 122 [INFO]    [select_level_agg] 
num: CREDIT_TERM
method: std
['SK_ID_CURR'] 
2018-07-21 15:36:05,670 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'CREDIT_TERM_std@['SK_ID_CURR']'], dtype='object') 
2018-07-21 15:36:05,670 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'CREDIT_TERM_std@['SK_ID_CURR']'], dtype='object') 
2018-07-21 15:36:05,750 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 15:36:05,750 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 15:36:05,771 logger 29 [INFO]    [make_npy] 0.000000     2597
0.707107     2248
1.414214     1556
2.121320     1219
2.828427     1204
3.535534     1068
4.242641     1038
4.949747      827
8.485281      705

2018-07-21 15:36:06,074 logger 122 [INFO]    [select_level_agg] 
num: CREDIT_TERM
method: min
['SK_ID_CURR'] 
2018-07-21 15:36:06,074 logger 122 [INFO]    [select_level_agg] 
num: CREDIT_TERM
method: min
['SK_ID_CURR'] 
2018-07-21 15:36:06,174 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'CREDIT_TERM_min@['SK_ID_CURR']'], dtype='object') 
2018-07-21 15:36:06,174 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'CREDIT_TERM_min@['SK_ID_CURR']'], dtype='object') 
2018-07-21 15:36:06,253 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 15:36:06,253 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 15:36:06,267 logger 29 [INFO]    [make_npy] 6.0      20962
3.0      14308
10.0     13409
5.0      12417
4.0      12263
1.0      11257
2.0       9214
12.0      8210
7.0       7952
11.0      7311
9.0       6

2018-07-21 15:36:06,538 logger 122 [INFO]    [select_level_agg] 
num: DAYS_CREDIT
method: mean
['SK_ID_CURR'] 
2018-07-21 15:36:06,538 logger 122 [INFO]    [select_level_agg] 
num: DAYS_CREDIT
method: mean
['SK_ID_CURR'] 
2018-07-21 15:36:06,644 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'DAYS_CREDIT_mean@['SK_ID_CURR']'], dtype='object') 
2018-07-21 15:36:06,644 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'DAYS_CREDIT_mean@['SK_ID_CURR']'], dtype='object') 
2018-07-21 15:36:06,728 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 15:36:06,728 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 15:36:06,748 logger 29 [INFO]    [make_npy] -90.000000    2539
-12.000000    1416
-11.000000    1390
-24.000000    1361
-16.000000    1345
-23.000000    1341
-20.000000    1340
-13.000000    1340
-17.0

2018-07-21 15:36:07,190 logger 122 [INFO]    [select_level_agg] 
num: DAYS_CREDIT
method: max
['SK_ID_CURR'] 
2018-07-21 15:36:07,190 logger 122 [INFO]    [select_level_agg] 
num: DAYS_CREDIT
method: max
['SK_ID_CURR'] 
2018-07-21 15:36:07,356 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'DAYS_CREDIT_max@['SK_ID_CURR']'], dtype='object') 
2018-07-21 15:36:07,356 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'DAYS_CREDIT_max@['SK_ID_CURR']'], dtype='object') 
2018-07-21 15:36:07,517 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 15:36:07,517 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 15:36:07,536 logger 29 [INFO]    [make_npy] -3.0     8351
-6.0     8148
-4.0     7971
-5.0     7961
-2.0     7934
-7.0     7418
-1.0     7260
-8.0     6843
-9.0     6254
-10.0    5868
-11.0    5607
-12.0  

...

2018-07-21 15:36:09,414 logger 29 [INFO]    [make_npy] -80.0     19202
-77.0      2300
-33.0      2238
-78.0      2222
-30.0      2155
-76.0      2150
-31.0      2140
-79.0      2127
-34.0      2108
-32.0      2097
 0.0       2086
-75.0      2058
-38.0      2036
-36.0      2024
-40.0      2001
-39.0      1926
-35.0      1918
-73.0      1900
-37.0      1877
-74.0      1840
-25.0      1814
-71.0      1800
-70.0      1782
-72.0      1762
-26.0      1758
-29.0      1753
-28.0      1736
-42.0      1723
-23.0      1722
-41.0      1709
-21.0      1698
-24.0      1697
-22.0      1697
-27.0      1686
-18.0      1685
-17.0      1683
-14.0      1666
-20.0      1654
-16.0      1651
-15.0      1643
-44.0      1627
-19.0      1608
-45.0      1606
-48.0      1599
-12.0      1595
-46.0      1594
-11.0      1590
-10.0      1583
-43.0      1576
-49.0      1574
          ...  
 70.0        10
 74.0         9
 75.0         9
-91.0         8
 98.0         8
 95.0         8
 67.0         8
 105.0        8
 

2018-07-21 15:36:09,953 logger 29 [INFO]    [make_npy] -80.000000    2766
-31.000000    1561
-17.000000    1560
-30.000000    1555
-14.000000    1540
-18.000000    1534
-16.000000    1512
-15.000000    1510
-22.000000    1504
-13.000000    1504
-12.000000    1504
-10.000000    1494
-21.000000    1488
-20.000000    1471
-11.000000    1468
-9.000000     1463
-32.000000    1455
-23.000000    1443
-25.000000    1440
-33.000000    1433
-7.000000     1429
-19.000000    1427
-26.000000    1427
-24.000000    1415
-29.000000    1410
-27.000000    1409
-8.000000     1400
-28.000000    1369
-6.000000     1361
-34.000000    1345
-5.000000     1338
-4.000000     1328
-36.000000    1276
-2.000000     1265
-3.000000     1243
-35.000000    1219
-38.000000    1214
-37.000000    1201
-40.000000    1191
-1.000000     1191
-39.000000    1187
-41.000000    1053
-42.000000    1035
-45.000000     937
 0.000000      920
-43.000000     906
-44.000000     884
-46.000000     881
-47.000000     770
-49.000000    

2018-07-21 15:36:10,525 logger 29 [INFO]    [make_npy] -1.0     6731
-2.0     6609
-3.0     6270
-4.0     6017
-5.0     5870
-6.0     5631
 0.0     5537
-7.0     5453
-8.0     5168
-9.0     4942
-10.0    4704
-11.0    4416
-12.0    3899
-13.0    3624
-14.0    3570
-15.0    3361
-16.0    3102
-17.0    3066
-18.0    2773
-80.0    2766
-19.0    2465
-20.0    2367
-21.0    2212
-22.0    2143
-23.0    2082
-24.0    2023
-30.0    1939
-25.0    1912
-26.0    1775
-31.0    1744
-27.0    1739
-32.0    1652
-28.0    1608
-29.0    1590
-33.0    1561
-34.0    1463
-36.0    1220
-35.0    1203
-37.0    1117
-38.0    1103
-39.0    1060
-40.0    1053
-42.0     815
-41.0     813
-43.0     733
-44.0     712
-45.0     709
-46.0     662
-48.0     635
-47.0     611
-49.0     609
-51.0     585
-50.0     583
-52.0     561
-53.0     489
-54.0     489
-55.0     464
-56.0     448
-71.0     444
-57.0     428
-59.0     424
-75.0     424
-77.0     422
-78.0     415
-58.0     413
-70.0     412
-72.0     412
-60.0  

2018-07-21 15:36:11,100 logger 122 [INFO]    [select_level_agg] 
num: DAYS_CREDIT_UPDATE
method: mean
['SK_ID_CURR'] 
2018-07-21 15:36:11,100 logger 122 [INFO]    [select_level_agg] 
num: DAYS_CREDIT_UPDATE
method: mean
['SK_ID_CURR'] 
2018-07-21 15:36:11,207 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'DAYS_CREDIT_UPDATE_mean@['SK_ID_CURR']'], dtype='object') 
2018-07-21 15:36:11,207 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'DAYS_CREDIT_UPDATE_mean@['SK_ID_CURR']'], dtype='object') 
2018-07-21 15:36:11,294 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 15:36:11,294 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 15:36:11,314 logger 29 [INFO]    [make_npy]  0.000000     15430
-1.000000      5672
-62.000000     3842
-2.000000      2494
-3.000000      1951
-4.000000      1888
-5.000000

2018-07-21 15:36:11,678 logger 122 [INFO]    [select_level_agg] 
num: DAYS_CREDIT_UPDATE
method: max
['SK_ID_CURR'] 
2018-07-21 15:36:11,678 logger 122 [INFO]    [select_level_agg] 
num: DAYS_CREDIT_UPDATE
method: max
['SK_ID_CURR'] 
2018-07-21 15:36:11,838 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'DAYS_CREDIT_UPDATE_max@['SK_ID_CURR']'], dtype='object') 
2018-07-21 15:36:11,838 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'DAYS_CREDIT_UPDATE_max@['SK_ID_CURR']'], dtype='object') 
2018-07-21 15:36:11,960 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 15:36:11,960 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 15:36:11,971 logger 29 [INFO]    [make_npy]  0.0     99322
-1.0     19312
-2.0      4610
-62.0     3842
-3.0      2919
-4.0      2439
-5.0      2157
-6.0      1798
-7.0      162

In [46]:
def bureau_credit_agg(base, df):

    df = diff_feature(df=df, first='AMT_CREDIT_SUM', second='AMT_CREDIT_SUM_DEBT')
    
#     num_list = ['AMT_CREDIT_SUM_diff_AMT_CREDIT_SUM_DEBT@']
    num_list = ['AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_diff_AMT_CREDIT_SUM_DEBT@','AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'AMT_ANNUITY', 'LOAN_MONTHLY_FACT', 'LOAN_MONTHLY']
    num_list = income_list
    select_level_agg(base=base, data=df, ignore_list=ignore_features, 
                     level=unique_id, method_list=method_list, num_list=num_list, prefix=prefix)
    
prefix='b_ACT_3YEAR_'
prefix='b_CLO_after_REG_'
# prefix='b_'

if prefix.count('_HALF'):
    data = extract_term(df=ab, term=-6)
elif prefix.count('_YEAR'):
    data = extract_term(df=ab, term=-12)
elif prefix.count('_2YEAR'):
    data = extract_term(df=ab, term=-24)
elif prefix.count('_3YEAR'):
    data = extract_term(df=ab, term=-36)
elif prefix.count('_REG'):
    data = ab.query('DAYS_CREDIT > DAYS_REGISTRATION')
elif prefix.count('_PUBLISH'):
    data = ab.query('DAYS_CREDIT > DAYS_ID_PUBLISH')
else:
    data = ab.copy()
    
data['DAYS_ENDDATE'].fillna(999, inplace=True)

' ACTIVE '
if prefix.count('ACT'):
    data = data.query('DAYS_ENDDATE>0')
elif prefix.count('CLO'):
    ' CLOSED '
    data = data.query('DAYS_ENDDATE<=0')

# df = data.groupby(unique_id)['LOAN_MONTHLY'].sum().reset_index()
# df = df.merge(data.groupby(unique_id)['AMT_INCOME_TOTAL'].max().reset_index(), on=unique_id, how='inner')
# df = division_feature(df=df, first='LOAN_MONTHLY', second='AMT_INCOME_TOTAL', sort=0)
# make_feature(df=df, prefix=prefix)
    
bureau_credit_agg(base=base, df=data)
del data

/mnt/c/Users/gixo/go/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
2018-07-21 16:13:40,972 logger 122 [INFO]    [select_level_agg] 
num: AMT_INCOME_TOTAL_div_AMT_CREDIT_MAX_OVERDUE@
method: sum
['SK_ID_CURR'] 
2018-07-21 16:13:40,972 logger 122 [INFO]    [select_level_agg] 
num: AMT_INCOME_TOTAL_div_AMT_CREDIT_MAX_OVERDUE@
method: sum
['SK_ID_CURR'] 
../../../github/module/feature_engineering.py:26: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  result = df.groupby(level)[feature].agg({'tmp': {method}})
2018-07-21 16:13:41,266 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'AMT_INCOME_TOTAL_di

2018-07-21 16:13:41,586 logger 29 [INFO]    [make_npy] 2500.000000     27
25.000000       22
12.500000       18
1500.000000     18
2000.000000     18
5000.000000     18
50.000000       17
30000.000000    16
20.000000       16
500.000000      16
7500.000000     15
10.000000       15
38.461538       15
41.666667       13
3000.000000     13
20000.000000    13
62.500000       13
33.333333       12
4000.000000     12
16.666667       12
66.666667       12
40.000000       12
1666.666667     11
100.000000      11
1000.000000     11
156.250000      11
55.555556       11
600.000000      11
22.727273       10
10000.000000    10
31.250000       10
15000.000000     9
333.333333       9
25000.000000     9
3333.333333      9
125.000000       9
3500.000000      9
750.000000       9
83.333333        9
26.315789        9
60.000000        8
31.914894        8
27.777778        8
57.692308        8
37.037037        8
50000.000000     8
19.607843        8
428.571429       8
250.000000       8
20.833333     

2018-07-21 16:13:42,029 logger 29 [INFO]    [make_npy] 2500.000000     36
50.000000       34
2000.000000     31
25.000000       31
5000.000000     25
20.000000       25
1500.000000     24
62.500000       23
12.500000       23
31.250000       21
500.000000      20
41.666667       20
3000.000000     20
38.461538       19
30000.000000    18
100.000000      18
4000.000000     18
20000.000000    18
7500.000000     17
55.555556       16
10.000000       16
33.333333       16
1000.000000     16
250.000000      15
66.666667       15
1666.666667     15
16.666667       15
40.000000       15
125.000000      14
10000.000000    14
22.727273       14
750.000000      14
60.000000       14
83.333333       14
45.454545       13
15000.000000    13
833.333333      12
28.571429       12
26.315789       12
600.000000      12
1250.000000     12
4500.000000     12
21.739130       12
25000.000000    11
37.037037       11
3333.333333     11
3500.000000     11
57.692308       11
200.000000      11
156.250000    

2018-07-21 16:13:42,477 logger 29 [INFO]    [make_npy] 0.000000     1565
2.000000      571
1.000000      546
2.500000      332
1.666667      315
0.500000      262
1.500000      258
1.250000      258
3.333333      251
5.000000      246
4.000000      244
3.000000      210
1.428571      198
1.200000      185
0.666667      185
0.600000      177
1.333333      169
0.750000      167
0.800000      157
0.833333      148
2.857143      145
0.400000      139
1.111111      129
0.333333      129
6.000000      128
0.857143      127
1.600000      121
0.200000      120
2.400000      116
0.250000      115
6.666667      107
2.222222      102
10.000000      98
2.666667       92
0.714286       91
8.000000       84
3.750000       79
0.571429       78
0.428571       78
0.166667       77
0.300000       76
0.625000       76
1.400000       76
2.333333       75
1.714286       74
3.500000       72
0.100000       72
7.500000       68
0.375000       68
1.800000       67
             ... 
1.491834        1
1.068038 

2018-07-21 16:13:42,982 logger 29 [INFO]    [make_npy] 0.000000    1765
0.707107      71
0.353553      52
1.414214      48
0.176777      30
1.060660      23
0.883883      21
2.121320      19
3.535534      19
0.471405      17
2.828427      16
0.088388      15
0.530330      15
0.424264      14
0.848528      14
0.265165      13
1.767767      13
2.474874      13
2.357023      13
0.441942      13
1.178511      12
1.010153      12
0.942809      12
0.565685      12
0.141421      12
0.989949      12
0.565685      12
1.697056      11
0.141421      11
0.235702      11
0.589256      11
0.141421      11
0.282843      11
0.707107      11
0.471405      10
0.589256      10
0.235702      10
1.237437      10
1.010153       9
1.272792       9
0.212132       9
1.414214       9
0.404061       9
1.555635       9
0.094281       9
0.202031       9
0.777817       8
1.979899       8
0.606092       8
1.571348       8
            ... 
0.623059       1
0.167874       1
1.017226       1
0.065587       1
0.118747  

...

2018-07-21 16:13:46,177 logger 29 [INFO]    [make_npy] 0.000000         134959
2500.000000           6
40000.000000          6
5000.000000           6
25000.000000          5
50000.000000          4
20000.000000          4
30000.000000          4
750.000000            3
666.666667            3
16666.666667          3
2000.000000           3
15000.000000          3
4000.000000           3
8750.000000           3
35000.000000          3
6250.000000           3
12000.000000          3
17500.000000          2
3333.333333           2
7000.000000           2
921.052632            2
51724.137931          2
9000.000000           2
512.820513            2
1250.000000           2
60000.000000          2
28000.000000          2
2727.272727           2
500.000000            2
3076.923077           2
0.554160              2
555.555556            2
227.272727            2
1666.666667           2
437.500000            2
8333.333333           2
7142.857143           2
1363.636364           2
0.664992 

2018-07-21 16:13:46,632 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 16:13:46,632 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 16:13:46,640 logger 29 [INFO]    [make_npy] 1.102446        1
22.082931       1
7065.194832     1
31.811843       1
151.522882      1
12234.545296    1
174.865796      1
1.785234        1
2487.968304     1
262.000183      1
2.287347        1
3.043501        1
0.061837        1
Name: AMT_INCOME_TOTAL_div_AMT_CREDIT_SUM_OVERDUE@_std@['SK_ID_CURR'], dtype: int64 
2018-07-21 16:13:46,640 logger 29 [INFO]    [make_npy] 1.102446        1
22.082931       1
7065.194832     1
31.811843       1
151.522882      1
12234.545296    1
174.865796      1
1.785234        1
2487.968304     1
262.000183      1
2.287347        1
3.043501        1
0.061837        1
Name: AMT_INCOME_TOTAL_div_AMT_CREDIT_SUM_OVERDUE@_std@['SK_ID_CURR'], dtype: int64 
2018-07-21 16:13:46,674 logger 122 [INFO]    [select_level_agg] 
num

2018-07-21 16:13:47,114 logger 29 [INFO]    [make_npy] 2500.000000      6
5000.000000      6
40000.000000     6
25000.000000     5
50000.000000     4
20000.000000     4
30000.000000     4
15000.000000     4
2000.000000      3
35000.000000     3
750.000000       3
8750.000000      3
4000.000000      3
16666.666667     3
6250.000000      3
666.666667       3
1250.000000      3
12000.000000     3
2727.272727      2
0.554160         2
7000.000000      2
1666.666667      2
28000.000000     2
9000.000000      2
555.555556       2
60000.000000     2
437.500000       2
227.272727       2
921.052632       2
51724.137931     2
17500.000000     2
3076.923077      2
142.857143       2
3000.000000      2
1285.714286      2
3846.153846      2
16500.000000     2
250.000000       2
8333.333333      2
500.000000       2
3333.333333      2
512.820513       2
1428.571429      2
1363.636364      2
7142.857143      2
103.092784       2
0.664992         2
1000.000000      2
1500.000000      2
1097.560976   

2018-07-21 16:13:47,617 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 16:13:47,617 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 16:13:47,627 logger 29 [INFO]    [make_npy] 10.000000     101
5.000000       91
20.000000      78
25.000000      57
30.000000      54
6.000000       52
6.666667       48
4.000000       48
12.500000      48
50.000000      47
15.000000      46
40.000000      43
35.000000      40
7.500000       39
8.000000       38
13.333333      37
16.666667      37
8.333333       35
12.000000      33
7.000000       32
3.333333       27
6.250000       27
2.500000       26
11.666667      25
45.000000      24
9.000000       21
14.000000      19
3.000000       19
5.555556       19
60.000000      18
33.333333      17
4.166667       17
2.000000       15
26.666667      14
7.142857       14
22.000000      13
4.500000       13
5.833333       13
8.571429       12
3.500000       12
83.333333      12
17.500000      12
8.928

2018-07-21 16:13:47,888 logger 122 [INFO]    [select_level_agg] 
num: AMT_INCOME_TOTAL_div_AMT_ANNUITY@
method: max
['SK_ID_CURR'] 
2018-07-21 16:13:47,888 logger 122 [INFO]    [select_level_agg] 
num: AMT_INCOME_TOTAL_div_AMT_ANNUITY@
method: max
['SK_ID_CURR'] 
2018-07-21 16:13:47,986 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'AMT_INCOME_TOTAL_div_AMT_ANNUITY@_max@['SK_ID_CURR']'], dtype='object') 
2018-07-21 16:13:47,986 logger 127 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'AMT_INCOME_TOTAL_div_AMT_ANNUITY@_max@['SK_ID_CURR']'], dtype='object') 
2018-07-21 16:13:48,107 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 16:13:48,107 logger 130 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-21 16:13:48,121 logger 29 [INFO]    [make_npy] 10.000000     117
20.000000      94
5.000000       93
25.000000      76
3

In [43]:
prefix='b_'

data = ab.query("BUREAU_OVERDUE_CNT_ALL >0")
latest = data.groupby(unique_id)['DAYS_CREDIT'].max().reset_index()
latest['DAYS_CREDIT_2'] = latest['DAYS_CREDIT'] - 5
latest.drop('DAYS_CREDIT', axis=1, inplace=True)
data = data.merge(latest, on=unique_id, how='inner')
# data['filter'] = data['DAYS_CREDIT'] - data['DAYS_CREDIT_-2']
data = data.query('DAYS_CREDIT>=DAYS_CREDIT_2')
display(data[[unique_id, 'DAYS_CREDIT', 'TARGET']])
sys.exit()

data = data.merge(latest, on=[unique_id, 'DAYS_CREDIT', 'DAYS_ENDDATE'], how='inner').drop_duplicates()
print(data.shape)
print(data[unique_id].drop_duplicates().shape)
sys.exit()
    
data['DAYS_ENDDATE'].fillna(999, inplace=True)

' ACTIVE '
if prefix.count('ACT'):
    data = data.query('DAYS_ENDDATE>0')
elif prefix.count('CLO'):
    ' CLOSED '
    data = data.query('DAYS_ENDDATE<0')

df = data.groupby(unique_id)['LOAN_MONTHLY'].sum().reset_index()
df = df.merge(data.groupby(unique_id)['AMT_INCOME_TOTAL'].max().reset_index(), on=unique_id, how='inner')
df = division_feature(df=df, first='LOAN_MONTHLY', second='AMT_INCOME_TOTAL', sort=0)
make_feature(df=df, prefix=prefix)
    
# bureau_credit_agg(base=base, df=data)
del data

,SK_ID_CURR,DAYS_CREDIT,TARGET
0,179735,-71.0,0.0
1,114166,-12.0,1.0
4,154094,-8.0,0.0
6,127520,-44.0,0.0
8,299541,-8.0,0.0
9,210690,-19.0,1.0
14,208307,-38.0,0.0
17,208307,-40.0,0.0
23,257173,-38.0,1.0
25,351898,-11.0,NaN


SystemExit: 

/mnt/c/Users/gixo/go/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
def extract_same_during_annuity(row):

    remaing = row_list.copy()
    remaing.remove(row)
    
    ''' 
    tmp_resultは、ループで回ってきたrow_noを持ったレコードに絞ったDF。
    この後のremaing_loopでこのレコードと同じ期間を持ったレコードを絞り、UNIONしていく 
    '''
    tmp_result = bureau.query(f'row_no=={row}')[[unique_id, b_id, 'row_no']]
    tmp = bureau.query(f'row_no=={row}')[[unique_id, b_id, 'row_no', 'DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE']].rename(columns={'DAYS_CREDIT':'start', 'DAYS_CREDIT_ENDDATE':'end'})
    
    for rem in tqdm(remaing):
        tmp2 = bureau.query(f'row_no=={rem}')[[unique_id, b_id, 'row_no', 'DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE']].rename(columns={'DAYS_CREDIT':'start2', 'DAYS_CREDIT_ENDDATE':'end2', 'row_no':'same_term_row_no', b_id:'add_id'})
        
        ' rowの期間とremaingで回ってきた他の行の期間を比較する為、merge。 '
        df = tmp2.merge(tmp, on=unique_id, how='left')
        
        start = df['start'].values
        end = df['end'].values
        start2 = df['start2']
        end2 = df['end2']
        
        start2 = start2[start <= start2][start2 <= end].to_frame().reset_index()['index'].to_frame()
        end2 = end2[start <= end2][end2 <= end].to_frame().reset_index()['index'].to_frame()
        index = pd.concat([start2, end2], axis=0).drop_duplicates()
        index['include_flg'] = 1
        df.reset_index(inplace=True)
        same_term = df.merge(index, on='index', how='inner')[[unique_id, 'add_id', 'same_term_row_no']].rename(columns={'add_id':b_id, 'same_term_row_no':'row_no'})
        
        tmp_result = pd.concat([tmp_result, same_term], axis=0)
        
    # ここでbureauにmergeするのは、AMT_CREDITなどをつけるため
    target = bureau.merge(tmp_result, on=[unique_id, b_id, 'row_no'], how='inner')
    target['serial_no'] = row
    print(row)
    return target

In [11]:
def pararell_same_during_annuity(base, app, bureau):
    '''
    同時期に支払いの生じていたbureau_idをまとめ、月あたり支払額の合計を作る。
    延滞を起こしたbureauについて、同時期に支払っていた金額を求める。また、それをINCOMEで割る。
    同時期に無理な割合の支払額を持っていたのではないか？という仮説
    '''
    
    value = 'LOAN_MONTHLY'
#     value = 'PAYMENT_MONTHLY'
    ' DAYS_CREDIT_ENDDATEがNaNの場合、仮決めで3ヶ月後を完了日とする '
    bureau['ENDDATE_CREDIT'] = bureau['ENDDATE_CREDIT'].fillna(3)
    
    p_list = pararell_process(extract_same_during_annuity, row_list)
    
    # 同時期のセットが3つとすると、それが3回ループでやってくるが、重み付き平均チックなのが良いので3つとも積んで平均とる
    print(len(p_list))
    result = pd.concat(p_list, axis=0)
    serial = result['serial_no'].min()
    
    result.to_csv(f'../data/csv_feature/{value}_same_term_feature_{serial}.csv', index=False)
    
# row_list = list(bureau['row_no'].drop_duplicates())
row_list = list(range(1,11,1))
# row_list = list(range(11,78,1))
# pararell_same_during_annuity(base, app, bureau)

a = pd.read_csv('../data/csv_feature/LOAN_MONTHLY_same_term_feature_1.csv')
b = pd.read_csv('../data/csv_feature/LOAN_MONTHLY_same_term_feature_11.csv')
result = pd.concat([a,b], axis=0)
result.to_csv(f'../data/csv_feature/LOAN_MONTHLY_same_during_feature.csv', index=False)

In [18]:
bureau_num = [
    'AMT_CREDIT_MAX_OVERDUE'
    ,'AMT_CREDIT_SUM'
    ,'AMT_CREDIT_SUM_DEBT'
    ,'AMT_CREDIT_SUM_LIMIT'
    ,'AMT_CREDIT_SUM_OVERDUE'
    ,'AMT_ANNUITY'
    ,'DAYS_CREDIT'
    ,'DAYS_CREDIT_ENDDATE'
    ,'DAYS_CREDIT_UPDATE'
    ,'DAYS_ENDDATE_FACT'
    ,'CREDIT_DAY_OVERDUE'
    ,'LOAN_MONTHLY'
    ,'End_CREDIT_MONTHLY'
]

# bureau_num = [
#     'AMT_CREDIT_SUM_per_INCOME'
#     , 'AMT_ANNUITY_per_INCOME'
#     , 'LOAN_MONTHLY_per_INCOME'
# ]

method_list = [
    'sum', 
    'mean'
    , 'std'
    , 'max'
    , 'min'
]

' 延滞してるレコードのみに絞って集計 '
def extarct_record(df):
#     df = df.query('over_flg==1')
    df = df.query('progress_flg==1')
#     prefix = 'bc_over_'
    prefix = 'bc_prog_'
    
    for num in bureau_num:
    
        for method in method_list:
            result = base_aggregation(df, unique_id, num, method, prefix)
            result = base.merge(result, on=unique_id, how='left')
            make_npy(result, ignore_features)

# extarct_record(bureau_clean)

In [28]:
bureau_num = [
    'AMT_CREDIT_MAX_OVERDUE'
    ,'AMT_CREDIT_SUM'
    ,'AMT_CREDIT_SUM_DEBT'
    ,'AMT_CREDIT_SUM_LIMIT'
    ,'AMT_CREDIT_SUM_OVERDUE'
    ,'AMT_ANNUITY'
    ,'DAYS_CREDIT'
    ,'DAYS_CREDIT_ENDDATE'
    ,'DAYS_CREDIT_UPDATE'
    ,'DAYS_ENDDATE_FACT'
    ,'CREDIT_DAY_OVERDUE'
    ,'LOAN_MONTHLY'
    ,'END_CREDIT_MONTHLY'
]

bureau_num = [
    'AMT_CREDIT_SUM_per_INCOME'
    , 'AMT_ANNUITY_per_INCOME'
    , 'LOAN_MONTHLY_per_INCOME'
]

method_list = [
    'sum', 
    'mean'
    , 'std'
    , 'max'
    , 'min'
]

' 延滞or現在の契約のレコードでそれぞれ集計し、差と比をとる'
def extarct_record(df):
    df_over = df.query('over_flg==1')
    df_prog = df.query('progress_flg==1')
#     prefix = 'bc_over_'
    prefix = 'bc_diff_over-prog_'
#     prefix = 'bc_ratio_over-prog_'
    
    for num in bureau_num:
    
        for method in method_list:
            result_over = base_aggregation(df_over, unique_id, num, method, prefix)
            result_over = base.merge(result_over, on=unique_id, how='left')
            
            result_prog = base_aggregation(df_prog, unique_id, num, method, prefix)
            result_prog = base.merge(result_prog, on=unique_id, how='left')
            
            feature_name = [col for col in result_over.columns if col.count('@')][0]
            result_over[feature_name] = result_over[feature_name] - result_prog[feature_name]
#             result_over[feature_name] = result_over[feature_name] / result_prog[feature_name]
            
            make_npy(result_over, ignore_features)

extarct_record(bureau_app)

../../../github/module/feature_engineering.py:56: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  result = df.groupby(level)[feature].agg({'tmp': {method}})


(356255, 2)
(356255, 2)
(356255, 2)
(356255, 2)
(356255, 2)
(356255, 2)
(356255, 2)
(356255, 2)
(356255, 2)
(356255, 2)
(356255, 2)
(356255, 2)
(356255, 2)
(356255, 2)
(356255, 2)
